In [143]:
import cv2
import numpy as np
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [215]:
# Load image, grayscale, and adaptive threshold
image = cv2.imread('..\images\puzzle.JPG')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 0)
thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,57,5)

cv2.imshow('thresh', thresh)


In [216]:
# Retrieve largest area in the image
cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

max_area = 0
c = 0
for i in cnts:
        area = cv2.contourArea(i)
        if area > 1000:
                if area > max_area:
                    max_area = area
                    best_cnt = i
                    image = cv2.drawContours(image, cnts, c, (0, 255, 0), 3)
        c+=1

In [217]:
mask = np.zeros((thresh.shape),np.uint8)
cv2.drawContours(mask,[best_cnt],0,255,-1)
cv2.drawContours(mask,[best_cnt],0,0,2)
#cv2.imshow("mask", mask)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [218]:
cv2.imshow("mask", mask)

In [219]:
out = np.zeros_like(thresh)
out[mask == 255] = gray[mask == 255]
cv2.imshow("New image", out)

In [172]:
out.shape

(600, 493)

In [220]:
approx = cv2.approxPolyDP(best_cnt, 0.02 * cv2.arcLength(best_cnt, True), True)

In [221]:
len(approx)

4

In [222]:
src = np.squeeze(approx, axis=1)
src = np.array(src,np.float32)

In [223]:
def width_and_height(approx):
    x_coord = list(approx[:,0][:,0])
    y_coord = list(approx[:,0][:,1])
    width = max(x_coord) - min(x_coord)
    height = max(y_coord) - min(y_coord)

    return width, height

In [224]:
width, height = width_and_height(approx)

In [225]:
warp_points = np.float32([[0,0],[0,height],[width,height],[width,0]])

In [226]:
matrix = cv2.getPerspectiveTransform(src,warp_points)

In [227]:
output = cv2.warpPerspective(out, matrix, (width,height))

In [233]:
cv2.imshow("output",output)

In [229]:
testOutput = output.copy()

In [154]:
threshOutput = cv2.adaptiveThreshold(testOutput,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,57,5)

In [149]:
threshOutput = cv2.adaptiveThreshold(testOutput,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,57,5)
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (int(width//1.1),1))
detected_lines = cv2.morphologyEx(threshOutput, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(threshOutput, [c], -1, (0,0,0), 2)
cv2.imshow("testOutput",threshOutput)

In [242]:
def image_crop(img,width,height):
    image_list = []
    box_width = width // 9
    box_height = height // 9
    for col in range(9):
        h = col * box_height
        for row in range(9):
            w = row * box_width
            image_list.append(img[h + int(0.1*box_height):int(h + box_height // 1.1) , w + int(0.3*box_width):int(w + box_width // 1.1)])
    return image_list

In [243]:
image_list = image_crop(output,width,height)

In [245]:
cv2.imshow("digit",image_list[0])

In [108]:
from keras.models import load_model

In [109]:
model = load_model('../model/mnist.h5')

In [262]:
def predict_number(img,model,debug=False):
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    #thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    _, cnts, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) != 0:
        c = max(cnts, key=cv2.contourArea)
        mask = np.zeros(thresh.shape,dtype='uint8')
        cv2.drawContours(mask,[c],0,255,0)
        
        (h,w) = img.shape
        percentFilled = cv2.countNonZero(mask) / float(w * h)

        if percentFilled < 0.03:
            return 0

        digit = cv2.bitwise_and(thresh, thresh, mask=mask)
        kernel = np.ones((3,3), np.uint8) 
        digit = cv2.morphologyEx(digit, cv2.MORPH_CLOSE, kernel)

        if debug:
            cv2.imshow("digit",digit)
            cv2.waitKey(0)

        #digit = cv2.threshold(output, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        digit = cv2.resize(digit, (28,28))
        digit = digit.reshape((1,28,28,1))
        digit = digit.astype("float")
        digit /= 255.0
        
        result = model.predict_classes(digit)
        result = result.tolist()
    else:
        result = 0
    return result

In [263]:
puzzle = []
for img in image_list:
    num = predict_number(img,model)
    puzzle.append(num)

In [264]:
predict_number(image_list[8],model,debug=True)

[8]

In [260]:
puzzle

[0,
 0,
 0,
 0,
 0,
 0,
 [3],
 [9],
 [8],
 [7],
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 [2],
 0,
 [3],
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 [9],
 [8],
 [7],
 0,
 [3],
 0,
 0,
 [3],
 0,
 [8],
 0,
 0,
 [9],
 0,
 [4],
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 [3],
 0,
 0,
 0,
 0,
 [5],
 0,
 0,
 0,
 0,
 0,
 [5],
 0,
 [2],
 0,
 [7],
 0,
 0,
 [9],
 0,
 0,
 0,
 [7],
 0,
 [4],
 [1],
 0]

In [27]:
out = model.predict_classes(np.expand_dims(crop_img2, axis=0))

In [ ]:
predict_number(image_list[23],model,debug=True)

In [135]:
len(cnts)
max(cnts,key=cv2.contourArea)

array([[[ 0,  0]],

       [[ 0, 51]],

       [[51, 51]],

       [[51,  0]]], dtype=int32)